## Second Assignment: Segmenting and Clustering Neighborhoods in Toronto

*For answer to the third question of the assignment skip to In[12]

First step: Install packages and import libraries

In [1]:
!conda install -c conda-forge wikipedia --yes
import pandas as pd
import wikipedia as wp
import numpy as np

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    wikipedia: 1.4.0-py35_0 conda-forge

wikipedia-1.4. 100% |################################| Time: 0:00:00  14.14 MB/s


Second step: Retieve table from html source and format to read fisrt row as header

In [2]:
#Get the html source
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]
df.to_csv('beautifulsoup_pandas.csv',header=0,index=False)
# Transform first column into header
df.columns=['PostalCode', 'Borough', 'Neighborhood']
df.drop(df.index[0], inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


Third step: drop rows with borough not assigned

In [3]:
df=df[df.Borough != 'Not assigned']
df.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


Third step: Group rows with identical postal code. First, in order to verify hoy many rows we should have in the end I used the group by, count() method by PostalCode

In [4]:
check=df.groupby('PostalCode').count()
check.shape

(103, 2)

This shows us there is a total of 103 different postal codes in the data frame
Then we group by postal code, but now we use the join function in order to mix different neighborhoods separated by a comma

In [5]:
new_df=df.groupby(['PostalCode', 'Borough'], as_index=False).agg(lambda x: ', '.join(set(x.astype(str))))
new_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, West Hill, Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Now we check that the total amount of rows is the same as the prdeicted one of 103

In [6]:
new_df.shape

(103, 3)

We can see that it is the same number of rows, so even though the order of the rows has changed the result is still correct

Fourth step: Replace not assigned values for neighborhood with value from borough

In [7]:
final_df=new_df
final_df['Neighborhood'].where(final_df.Neighborhood!='Not assigned', final_df.Borough, inplace=True)
final_df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, West Hill, Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Kennedy Park, Ionview"
7,M1L,Scarborough,"Oakridge, Clairlea, Golden Mile"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


To finish up, we check that there are no rows containing a Not assigned neighborhoood, that could have been joined with other neighborhoods when grouping by postal code

In [8]:
final_df[final_df['Neighborhood'].str.contains("assigned")]

,PostalCode,Borough,Neighborhood


And we verify once more the shaoe of the final dataframe

In [9]:
final_df.shape

(103, 3)

Second question of the assignment

First step is to retrieve the latitude and longitude of each postal code and assigning it in new columns labeled 'Latitude' and 'Longitude' to the data frame. Since I couldn't retrieve the latitude and longitude through the geocoder library, I will use the csv file and name it geo_data

In [10]:
geo_data=pd.read_csv('http://cocl.us/Geospatial_data')
geo_data.rename(columns={'Postal Code':'PostalCode', 'Latitude':'Latitude', 'Longitude':'Longitude'}, inplace=True)
geo_data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Then I merged the data frames using the merge method, to assign the correct latitude and longitude to each postal code

In [11]:
merged_df=pd.merge(final_df, geo_data, how='left', on='PostalCode')
merged_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, West Hill, Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Kennedy Park, Ionview",43.727929,-79.262029
7,M1L,Scarborough,"Oakridge, Clairlea, Golden Mile",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


Third question of the assignment

For the following steps, first i will import the libraries required to get the information on these neighborhoods, cluster them and display the resulting maps:

In [12]:
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  45.44 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  32.00 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  34.16 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  32.54 MB/s


Then I move on to visualizing Toronto and every postal code, in order to decide whether to use all of the neighborhoodsor just a fraction. I obtained  the coordinates for Toronto online: -79.4163000, 43.7001100

In [13]:
latitude = 43.7001100
longitude = -79.4163000

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Borough'], merged_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

I will keep every nieghborhood, since there doesn't seem to be a natural limit between them. 

In order to begin the process of exploring them, i define my foursquare credentials in the next step

In [14]:
CLIENT_ID = 'F2ELEML3GET2RAUDGCNQMG3Z3YVOR2DDW5YSADCUHOV0JA5W'
CLIENT_SECRET = 'LUNTF0C3EXJN45I33DCG1WWHA21GID5MGG3RPKFPH03NMXZI' 
VERSION = '20180605'
radious=500
LIMIT=100

I define a function to explore every neighborhood given on a dataframe as input and then I run it for the dataframe with Toronto's nieghborhoods

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
toronto_venues = getNearbyVenues(names=merged_df['Neighborhood'],
                                   latitudes=merged_df['Latitude'],
                                   longitudes=merged_df['Longitude']
                                  )

Rouge, Malvern
Rouge Hill, Port Union, Highland Creek
Guildwood, West Hill, Morningside
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Kennedy Park, Ionview
Oakridge, Clairlea, Golden Mile
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Scarborough Town Centre, Dorset Park, Wexford Heights
Wexford, Maryvale
Agincourt
Sullivan, Tam O'Shanter, Clarks Corners
Agincourt North, Steeles East, Milliken, L'Amoreaux East
L'Amoreaux West, Steeles West
Upper Rouge
Hillcrest Village
Oriole, Fairview, Henry Farm
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Downsview North, Bathurst Manor, Wilson Heights
York University, Northwood Park
Downsview East, CFB Toronto
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The D

Now we have every venue discovered in these neighborhoods in the toronto_venues data frame. In this case I wil only show the top 5 rows as a representetion of the complete dataframe

In [17]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
4,"Guildwood, West Hill, Morningside",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


In [18]:
toronto_venues.groupby('Neighborhood').count().shape

(100, 6)

Then I group each neighborhood in order to get the top 5 categories of venue they have, so I can in the end cluster them based on the categories that are popular on each neighborhood

First I create a dataframe that contains each Neighborhood that returned a venue (100 out of the original 103) and the frecuency of each category for that neighborhood

In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
manhattan_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Agincourt North, Steeles East, Milliken, L'Amo...",0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Bayview Village,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Berczy Park,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Business Reply Mail Processing Centre 969 Eastern,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632
6,Caledonia-Fairbanks,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000
7,Canada Post Gateway Processing Centre,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Cedarbrae,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Central Bay Street,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.012346,0.000000,0.000000,0.000000,0.000000,0.012346,0.000000,0.000000,0.012346


Then I create a function to sort venues in descending order

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

And I use that function to create a dataframe with the top5 category of venue on each of the 99 remaining neighborhoods

In [21]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Lounge,Clothing Store,Breakfast Spot,Skating Rink,Yoga Studio
1,"Agincourt North, Steeles East, Milliken, L'Amo...",Asian Restaurant,Playground,Park,Dumpling Restaurant,Diner
2,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Yoga Studio
3,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Steakhouse,Bakery
4,"Birch Cliff, Cliffside West",College Stadium,Café,General Entertainment,Skating Rink,Drugstore


Then i use k-means to cluster the neighborhoods

In [22]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Then I create a new dataframe that includes the cluster and the top 5 venues categories

In [23]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

toronto_merged = merged_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0.0,Bar,Moving Target,Construction & Landscaping,Yoga Studio,Dog Run
2,M1E,Scarborough,"Guildwood, West Hill, Morningside",43.763573,-79.188711,0.0,Mexican Restaurant,Pizza Place,Rental Car Location,Electronics Store,Breakfast Spot
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Yoga Studio,Dog Run,Doner Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Thai Restaurant


Finally I drop the rows that have no cluster by excluding those that have a Cluster_Labels value outside 0-4 range

In [24]:
tm=toronto_merged[(toronto_merged.Cluster_Labels>=0) & (toronto_merged.Cluster_Labels<5)]

Then I cast 'Cluster_Labels' as an integer so I can use it as an index in the next step

In [25]:
tm['Cluster_Labels']=tm['Cluster_Labels'].astype(int)
tm

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2,Fast Food Restaurant,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,Bar,Moving Target,Construction & Landscaping,Yoga Studio,Dog Run
2,M1E,Scarborough,"Guildwood, West Hill, Morningside",43.763573,-79.188711,0,Mexican Restaurant,Pizza Place,Rental Car Location,Electronics Store,Breakfast Spot
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Yoga Studio,Dog Run,Doner Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Thai Restaurant
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0,Business Service,Playground,Dumpling Restaurant,Diner,Discount Store
6,M1K,Scarborough,"East Birchmount Park, Kennedy Park, Ionview",43.727929,-79.262029,0,Department Store,Hobby Shop,Bus Station,Train Station,Convenience Store
7,M1L,Scarborough,"Oakridge, Clairlea, Golden Mile",43.711112,-79.284577,0,Bus Line,Bakery,Fast Food Restaurant,Intersection,Metro Station
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,0,Motel,American Restaurant,Dumpling Restaurant,Discount Store,Dog Run
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,0,College Stadium,Café,General Entertainment,Skating Rink,Drugstore


And then I use the folium library to map every neighborhood in a different color according to its cluster

In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tm['Latitude'], tm['Longitude'], tm['PostalCode'], tm['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters